<a href="https://colab.research.google.com/github/nerdk312/AMDIM_Decoder/blob/master/Pixel_CNNpp_V2_150220_Wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

In [0]:
!pip install wandb

     |████████████████████████████████| 1.4MB 2.9MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 460kB 20.0MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=f62b84a72cc07fb82169012771a24f4c694fdb8bfe2b5e99de7fd86f07052086
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=d019553807506bf3f0ae63547e655bd8c2dbb4a854e1974f86d0409b053f0721
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for shortuuid: filename=shortuuid-0.5.0-cp36-none-any.whl size=5497 sha256=f4de

In [0]:
!wandb login <API KEY>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import time
from __future__ import print_function
import pickle
import sys
#sys.path.append('/content/gdrive/My Drive/pixel-cnn-pp_V2_090220')

sys.path.append('/content/gdrive/My Drive/pixel-cnn-pp_V2_110220_Colab')

import wandb

import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.optim import lr_scheduler
from torchvision import datasets, transforms, utils
#from tensorboardX import SummaryWriter
from utils import * 
from model import * 
from PIL import Image

# Function to sample an image

In [0]:
def sample(model):
    model.train(False)
    data = torch.zeros(sample_batch_size, obs[0], obs[1], obs[2])
    data = data.cuda()
    for i in range(obs[1]):
        for j in range(obs[2]):
            data_v = Variable(data, volatile=True)
            out,_,_ = model(data_v, sample=True) # Nawid - Add to add 2 empty outputs as I do not require the log and the variance I believe
            out_sample = sample_op(out)
            data[:, :, i, j] = out_sample.data[:, :, i, j]
    return data

In [0]:
def reconstruct_image(model,data_loader):
    input, labels = iter(data_loader).next()
    model.train(False)
    input = input.cuda(async=True)
    input_var = Variable(input)
    out, _, _ = model(input_var) 
    out_sample =sample_op(out)
    return out_sample, input

# Model Setup

In [0]:
bc4 = False
args = {'data_dir':'data' if bc4 else '/content/gdrive/My Drive/pixel_CNN_data',
        'save_dir':'models',
        'dataset':'cifar',
        'print_every':50,
        'save_interval': 3,
        'load_params':None,
        'nr_resnet': 3,
        'nr_filters': 60,
        'nr_logistic_mix':10,
        'lr':0.0002,
        'lr_decay':0.999995,
        'batch_size':16,
        'max_epochs':100,
        'seed':1
}

if bc4: # Nawid-  Uses argparser if using bluecrystal
    parser = argparse.ArgumentParser()
    # data I/O
    parser.add_argument('-i', '--data_dir', type=str,
    default='data', help='Location for the dataset')
    parser.add_argument('-o', '--save_dir', type=str, default='models',
    help='Location for parameter checkpoints and samples')

    parser.add_argument('-d', '--dataset', type=str,
    default='cifar', help='Can be either cifar|mnist')
    parser.add_argument('-p', '--print_every', type=int, default=50,
    help='how many iterations between print statements')
    parser.add_argument('-t', '--save_interval', type=int, default=3,
    help='Every how many epochs to write checkpoint/samples?')
    parser.add_argument('-r', '--load_params', type=str, default=None,
    help='Restore training from previous model checkpoint?')
    # model
    parser.add_argument('-q', '--nr_resnet', type=int, default=3,
    help='Number of residual blocks per stage of the model')
    parser.add_argument('-n', '--nr_filters', type=int, default=60,
    help='Number of filters to use across the model. Higher = larger model.')
    parser.add_argument('-m', '--nr_logistic_mix', type=int, default=10,
    help='Number of logistic components in the mixture. Higher = more flexible model')
    parser.add_argument('-l', '--lr', type=float,
    default=0.0002, help='Base learning rate')
    parser.add_argument('-e', '--lr_decay', type=float, default=0.999995,
    help='Learning rate decay, applied every step of the optimization')
    parser.add_argument('-b', '--batch_size', type=int, default=32,
    help='Batch size during training per GPU')
    parser.add_argument('-x', '--max_epochs', type=int,
    default=100, help='How many epochs to run in total?')
    parser.add_argument('-s', '--seed', type=int, default=1,
    help='Random seed to use')
    command_args = parser.parse_args()

    args['data_dir'] = command_args.data_dir
    args['save_dir'] = command_args.save_dir
    args['dataset'] = command_args.dataset
    args['print_every'] = command_args.print_every
    args['save_interval'] = command_args.save_interval
    args['load_params'] = command_args.load_params
    args['nr_resnet'] = command_args.nr_resnet
    args['nr_filters'] = command_args.nr_filters
    args['nr_logistic_mix'] = command_args.nr_logistic_mix
    args['lr'] = command_args.lr
    args['lr_decay'] = command_args.lr_decay
    args['batch_size'] = command_args.batch_size
    args['max_epochs'] = command_args.max_epochs
    args['seed'] = command_args.seed

print(args['data_dir'])
wandb.init(entity="nerdk312", project="pytorch-PixelCNN",config=args)

torch.manual_seed(args['seed'])
np.random.seed(args['seed'])

model_name = 'pcnn_lr:{:.5f}_nr-resnet{}_nr-filters{}'.format(args['lr'], args['nr_resnet'], args['nr_filters'])

#model_folder = os.path.join('/content/gdrive/My Drive/pixel_cnn_runs', model_name)
#assert not os.path.exists(os.path.join('/content/gdrive/My Drive/pixel_cnn_runs', model_name)), '{} already exists!'.format(model_name)
#writer = SummaryWriter(log_dir= model_folder) # Nawid - Changed the name where the information is saved

#os.mkdir(model_folder) # Nawid - Used to make the folder when the summary writer is present

#epoch_models = os.path.join(model_folder,'models')
#images_models = os.path.join(model_folder,'images') 

#os.mkdir(epoch_models)
#os.mkdir(images_models)

sample_batch_size = 25
obs = (3, 64, 64) if 'STL10' in args['dataset'] else (3, 32, 32)

input_channels = obs[0]
desired_size = obs[1]

rescaling     = lambda x : (x - .5) * 2.
rescaling_inv = lambda x : .5 * x  + .5
kwargs = {'num_workers':1, 'pin_memory':True, 'drop_last':True}

if 'STL10' in args['dataset']:
    ds_transforms = transforms.Compose([transforms.CenterCrop(desired_size),transforms.ToTensor(), rescaling]) 
  
else:
    ds_transforms = transforms.Compose([transforms.ToTensor(), rescaling])

image_dim = desired_size


if 'cifar' in args['dataset']:
    train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(args['data_dir'], train=True, 
                                                              download=True, transform=ds_transforms), batch_size=args['batch_size'], shuffle=True, **kwargs)
    test_loader  = torch.utils.data.DataLoader(datasets.CIFAR10(args['data_dir'], train=False, 
                                                              transform=ds_transforms), batch_size=args['batch_size'], shuffle=True, **kwargs)
    loss_op   = lambda real, fake : discretized_mix_logistic_loss(real, fake)
    elbo_op = lambda mean, logvariance: elbo_loss(mean,logvariance)
    sample_op = lambda x : sample_from_discretized_mix_logistic(x, args['nr_logistic_mix'])

elif 'STL10' in args['dataset']:
    train_loader = torch.utils.data.DataLoader(datasets.STL10(args['data_dir'], split='train', 
                                                              download=True, transform=ds_transforms), batch_size=args['batch_size'], shuffle=True, **kwargs)
    test_loader  = torch.utils.data.DataLoader(datasets.STL10(args['data_dir'], split='test', 
                                                              transform=ds_transforms), batch_size=args['batch_size'], shuffle=True, **kwargs)
    loss_op   = lambda real, fake : discretized_mix_logistic_loss(real, fake)
    elbo_op = lambda mean, logvariance: elbo_loss(mean, logvariance)
    sample_op = lambda x : sample_from_discretized_mix_logistic(x, args['nr_logistic_mix'])

else :
    raise Exception('{} dataset not in {STL10, cifar10}'.format(args['dataset']))

/content/gdrive/My Drive/pixel_CNN_data


Files already downloaded and verified


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.cuda.empty_cache() # Nawid - I am not sure if this helps - hopefully it does

model = PixelCNN(nr_resnet=args['nr_resnet'], nr_filters=args['nr_filters'],
            input_channels=input_channels,image_size = desired_size, nr_logistic_mix=args['nr_logistic_mix']) # Nawid - Instantiates the model
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  #model = nn.DataParallel(model,) # Nawid - Separate across different GPUS
  model = nn.DataParallel(model, device_ids=[0, 1]) # Nawid - Specifying which GPUs to use to see if this makes a difference
model.to(device)

print('Model instantiated')

if args['load_params']:
    load_part_of_model(model, args['load_params'])
    print('model parameters loaded')

optimizer = optim.Adam(model.parameters(), lr=args['lr'])
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=args['lr_decay'])

torch.cuda.empty_cache() # Nawid - I am not sure if this helps - hopefully it does

print('starting training')
best_test_loss = 1e6 # Nawid - Set a very high initial maximum test loss so that the model will likely be higher than this


for epoch in range(args['max_epochs']):
    print('epoch_number',epoch)
    model.train(True)
    torch.cuda.synchronize()
    train_loss = 0.
    time_ = time.time()
    model.train()

    for batch_idx, (input,_) in enumerate(train_loader):
        input = input.cuda(non_blocking=True) # Nawid- Changed async= True to non_blocking= True as async gets depreciated
        #input = input.cuda(async=True)
        
        input = Variable(input)
        output,mu,logvar = model(input)
    
        loss = loss_op(input, output) + elbo_op(mu,logvar)  # Nawid - loss_op represents the discretised mixture of logistics loss function
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.data.item()#[0]
        
        if (batch_idx +1) % args['print_every'] == 0: 
            deno = args['print_every'] * args['batch_size'] * np.prod(obs) * np.log(2.)
            wandb.log({'train loss':(train_loss / deno)})
            print('loss : {:.4f}, time : {:.4f}'.format(
                (train_loss / deno), 
                (time.time() - time_)))
            train_loss = 0.
            time_ = time.time()
    

  # decrease learning rate
    scheduler.step()
    torch.cuda.empty_cache() # Nawid - This could possibly help with the lack of memory
  
    torch.cuda.synchronize()
    model.eval()
    test_loss = 0.
    total_test_ll_loss = 0. # Nawid - Separate log likelihood loss
    total_test_reg_loss = 0. # Nawid - Separate regularisation loss
  
    for batch_idx, (input,_) in enumerate(test_loader):
        torch.cuda.empty_cache() # Nawid - This could possibly help with the lack of memory
        #input = input.cuda(async=True)
        input = input.cuda(non_blocking=True)
        input_var = Variable(input) # Nawid - Should change to torch.no_grad !!!!
        output, mu, logvar = model(input_var) 
 
    
        test_ll_loss = loss_op(input_var, output) 
        test_reg_loss = elbo_op(mu,logvar)
    
        test_loss += test_ll_loss.item() + test_reg_loss.item()
        total_test_ll_loss += test_ll_loss.item()
        total_test_reg_loss += test_reg_loss.item()
    
        del test_ll_loss, test_reg_loss, output
    

    deno = batch_idx * args['batch_size'] * np.prod(obs) * np.log(2.)
    wandb.log({'test loss':(test_loss / deno), 'test loglikelihood loss':(total_test_ll_loss / deno),'test reg loss':(total_test_reg_loss / deno)})
    
    print('test loss : {:.5f}, log likelihood loss : {:.5f}, regularisation loss {:.5f}'.format(
        (test_loss/deno),
        (total_test_ll_loss / deno),
        (total_test_reg_loss / deno)))
  
    if (epoch + 1) % args['save_interval'] == 0: 
        torch.save(model.state_dict(), os.path.join(wandb.run.dir,'{}_{}.pth'.format(model_name, epoch))) # Nawid - Used to save the model in the wandb directory
        wandb.save(os.path.join(wandb.run.dir,'{}_{}.pth'.format(model_name, epoch))) # Nawid -  Used to save in the wandb directory -  Not sure if it saves twice or once at the moment

        '''
        print('sampling...')
        sample_t = sample(model)
        sample_t = rescaling_inv(sample_t)
        wandb.log({'sampled_image_{}'.format(epoch):wandb.Image(sample_t)})
        '''
        
        reconstruct_t, original_t = reconstruct_image(model,test_loader)
        reconstruct_t = rescaling_inv(reconstruct_t)
        original_t = rescaling_inv(original_t)

        wandb.log({'reconstructed_image_{}'.format(epoch):wandb.Image(reconstruct_t)})
        wandb.log({'original_image_{}'.format(epoch):wandb.Image(original_t)})
        #wandb.log({'sampled_image_{}'.format(epoch):wandb.Image(sample_t)})

In [0]:
import shutil
shutil.rmtree(os.path.join('/content/gdrive/My Drive/pixel_cnn_runs', model_name))

FileNotFoundError: ignored

# Practice code

In [0]:
model = PixelCNN(nr_resnet=args['nr_resnet'], nr_filters=args['nr_filters'], 
            input_channels=input_channels, nr_logistic_mix=args['nr_logistic_mix']) # Nawid - Instantiates the mode


reconstruct_t, original_t = reconstruct_image(model,test_loader)
reconstruct_t = rescaling_inv(reconstruct_t)
reconstruct_t = reconstruct_t.permute(0,2,3,1).detach().cpu().numpy()

original_t = rescaling_inv(original_t)
original_t = original_t.permute(0,2,3,1).detach().cpu().numpy()
single_reconstruct_t = reconstruct_t[0]
single_original_t = original_t[0]

Image.fromarray((single_reconstruct_t * 255).astype('uint8')).save(os.path.join(images_models,'{}_{}_{}.png'.format(model_name, 1,'reconstruction')))
Image.fromarray((single_original_t * 255).astype('uint8')).save(os.path.join(images_models,'{}_{}_{}.png'.format(model_name, 1,'original')))
'''
single_reconstruct_t = reconstruct_t[0]
single_original_t = original_t[0]
utils.save_image(single_reconstruct_t,os.path.join(images_models,'{}_{}_{}.png'.format(model_name, 1,'reconstruction')),nrow=1)
utils.save_image(single_original_t,os.path.join(images_models,'{}_{}_{}.png'.format(model_name, 1,'original')),nrow=1)
'''

In [0]:
'''
#torch.save(model.state_dict(), "model.h5")
#wandb.save('model.h5')

#torch.save(model.state_dict(),'{}_{}.pth'.format(model_name, epoch))
#wandb.save('{}_{}.pth'.format(model_name, epoch))

#wandb.save()
#torch.save(model.state_dict(), os.path.join(wandb.run.dir,'{}_{}.pth'.format(model_name, epoch))) # Nawid - Used to save the model in the wandb directory
'''